In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append(os.path.abspath("../../framework"))
sys.path.append(os.path.abspath("../../concrete"))
sys.path.append(os.path.abspath("../../snippet"))

In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation
from framework import  ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

# 10. Misc

## 12. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## 14. a function to load gain and bias from the trains over a training:

In [ ]:
def getGainAndBiasOverTrain(buildOrderIdGiven, timeSimulationGiven):

    concStore = ConcStore()
    agentFactory = ConcAgentFactory()

    y = np.random.randn(1, ConcEnvironment.nPv).astype(np.float32)
    anObservationSequence = ObservationSequence()
    anObservationSequence.add(ConcObservation(y))

    cur.execute('''
    Select
        count(*)
        From TrainLog
        Where buildOrderId = ?
        ''', (buildOrderIdGiven,))
    assert cur.fetchone()[0] > 0, "NO DATA WITH THE GIVEN BUILD ID"

    cur.execute('''
    Select 
        trainId
        , timeSimulation
        , timestamp
        From TrainLog
        Where buildOrderId = ?
        And timeSimulation = ?
        Order By timeSimulation
        ''', (buildOrderIdGiven, timeSimulationGiven, ))


    TimeSimulation = []
    Gain = []
    Bias = []

    (trainIdStr, timeSimulation, timestamp) = cur.fetchone()

    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(TrainId(trainIdStr))

    # restore agent
    agent = agentFactory.create(storeField.getBuildOrder())
    agent.loadFromMemento(storeField.getAgentMemento())
    agent(anObservationSequence)

    TimeSimulation.append(timeSimulation)

    gain = agent.gainP.weights[0].numpy() # (1, nMv)
    if len(agent.gainP.weights) > 1:
        bias = agent.gainP.weights[1].numpy() # (nMv)
    else:
        bias = np.nan
            
    weightOnError = storeField.getBuildOrder().weightOnError
    
    return gain, bias, weightOnError

# 20. Observe the trace of gain and bias over a traing

## 22. Choose a build order

In [ ]:
cur.execute('''
Select
    distinct buildOrderId
    From TrainLog
    Where buildOrderId like "InfiniteRewardSeries%"
    Order by timestamp
    ''')

buildOrderIdsToExport = [elm for elm, in cur.fetchall()]

## 24. Load the parameters => SKIPPED

## 26. Plot the trace of Gain and Bias along with the simulation time => SKIPPED

## 28. Export the dataset of gains

Choose a set of iterations over the training

In [ ]:
cur.execute('''
Select
    distinct TimeSimulation
    From TrainLog
    Where buildOrderId like "InfiniteRewardSeries%"
    Order by TimeSimulation
    ''')

timeSimulations = [elm for elm, in cur.fetchall()]

In [ ]:
timeSimulationsToExport = [timeSimulations[i] for i in np.linspace(1, len(timeSimulations)-1, 4).astype(np.int)]

Export the gain with respect to the pair of (training iteration, weightOnError)...

In [ ]:
txt = ["value,epoch,weightOnError,buildOrderId",]
for buildOrderIdGiven in buildOrderIdsToExport:
    for timeSimulationGiven in timeSimulationsToExport:
        try:
            gain, bias,  weightOnError = getGainAndBiasOverTrain(buildOrderIdGiven, timeSimulationGiven)
            txt.append("{0},{1},weightOnError={2},{3}".format(gain[0,0],timeSimulationGiven,weightOnError,buildOrderIdGiven))
        except:
            pass

with open("./tmp/gain.csv", "w") as fp:
    fp.write("\n".join(txt))